# Show compacted results from json



### Functions

In [29]:
import json
import numpy as np
import datetime

def print_total_times(ALL):
    train_time = []
    test_time = []
    datasets = list(ALL['datasets'].keys())
    for source in datasets:
        for target in datasets:
            if source == target:
                continue
            #concat train time from 3 steps
            train_time = train_time + np.ndarray.flatten(np.array(list(ALL['results'][source][target]['train']['time']))).tolist()
            test_time = test_time \
                + ALL['results'][source][target]['test']['source']['test']['time'] \
                + ALL['results'][source][target]['test']['target']['train']['time'] \
                + ALL['results'][source][target]['test']['target']['test']['time']
    #print(train_time)
    #print(test_time)
    tot_time = sum(train_time) + sum(test_time)
    tot_time = tot_time/(ALL['act_repetition'] +1) # se han hecho x ejecuciones de todo

    print("Train/Test time--all_combinations x1:  ", datetime.timedelta(seconds=tot_time))
    print("Train/Test time--all_combinations x5:  ", datetime.timedelta(seconds=tot_time*5))
    print("Train/Test time--all_combinations x10: ", datetime.timedelta(seconds=tot_time*10))
    print("")
    print("Total Train time: ", datetime.timedelta(seconds=sum(train_time)))
    print("Total Test time: ", datetime.timedelta(seconds=sum(test_time)))
    print("")

def print_mean_matrix(ALL):
    datasets = list(ALL['datasets'].keys())
    last_rep = ALL['act_repetition'] # finished
    if last_rep + 1 == ALL['parameters']['repeticiones']:
        last_rep += 1
    #last_rep += 1 ## QUITAR (solo sirve para mostrar matrices sin terminar)
    m = np.mean(ALL['matrix per repetition'][:last_rep], axis=0)
    std = np.std(ALL['matrix per repetition'][:last_rep], axis=0)
    print("Model Name: {}".format(ALL['parameters']['model_name']))
    print("Repetitions:", last_rep)
    print("Target IOU Matrix:", end='')
    print_matrix(m, std, datasets)

def print_matrix(m, std, datasets):
    print("\nsource\\target ", end='')
    for d in datasets:
        print("{:^17}".format(d), end='')
    print('\n', end='')
    for i, row in enumerate(m):
        print('{:<14}'.format(datasets[i]), end='')
        for j, e in enumerate(row):
            if j == i:
                print("{:^17}".format('-'*8), end='')
            else:
                if std[i,j] <= 0:
                    print("{:^17}".format( "{:.4f}".format(e)), end='')
                else:
                    print("{:^17}".format( "{:.4f} ±{:.4f}".format(e, std[i,j])), end='')
        print("\n", end='')
    print("")

def print_finished_matrix(ALL, step, metric, domain):
    datasets = list(ALL['datasets'].keys())
    last_rep = ALL['act_repetition'] # finished
    if last_rep + 1 == ALL['parameters']['repeticiones']:
        last_rep += 1
    mat = np.zeros((len(datasets),)*2).tolist()
    s = 1 if domain == 'source' else 2 # test, else train/test
    for i, source in enumerate(datasets):
        for j, target in enumerate(datasets):
            if source == target:
                mat[i][j] = [0,]*(last_rep*s)
                continue
            mse = []
            for set_ in ['train', 'test']:
                #print(ALL['results'][source][target]['test'][domain][set_].keys())
                #print(source,target,set_)
                mse = mse + list(ALL['results'][source][target]['test'][domain][set_][metric])
            mat[i][j] = (np.array(mse[step::3]).tolist())[:last_rep*s]
    mat = np.array(mat)
    #print(mat)
    m = np.mean(mat, axis=2)
    std = np.std(mat, axis=2)
    
    print("step{} {} {} Matrix:".format(step, domain.capitalize(), metric.upper()), end='')
    print_matrix(m, std, datasets)

## See results

In [30]:
json_path = './Results_YNet.json' # CHANGE THIS

f = open(json_path)
ALL = json.load(f)

print_mean_matrix(ALL)
#print_total_times(ALL)

Model Name: Att_YNet
Repetitions: 10
Target IOU Matrix:
source\target     Lucchi++            VNC          Kasthuri++    
Lucchi++          --------      0.4024 ±0.0401   0.6676 ±0.0203  
VNC            0.5360 ±0.0221      --------      0.3891 ±0.0415  
Kasthuri++     0.7036 ±0.0454   0.2520 ±0.0475      --------     



In [34]:
# If you want to explore other results (other steps etc.)

#print_finished_matrix(ALL, step=1, metric='mse', domain='target')
#print_finished_matrix(ALL, step=1, metric='mse', domain='source')
#print('_'*80+'\n')
#print_finished_matrix(ALL, step=2, metric='iou', domain='source')
#print_finished_matrix(ALL, step=0, metric='iou', domain='source')